<a href="https://colab.research.google.com/github/patelharsh94/HP-Nano-GPT/blob/main/HP_Nano_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nano GPT
Building a basic Chat GPT clone called Nano GTP which is trained off the "Tiny Shakespeare" dataset.  This GPT will understand the works of Shakespeare and reply to your prompts in a Shakespeareing manner.

This work references the video "**Let's build GPT: from scratch, in code, spelled out.**" found [here](https://www.youtube.com/watch?v=kCc8FmEb1nY&ab_channel=AndrejKarpathy).

This is a video by **Andrej Karpathy** one of the leading researchers and thought leaders of the AI/ML world.


In [2]:
# Getting the tiny shakespeare training data.

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-06-11 02:51:36--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2025-06-11 02:51:37 (22.8 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print(text[:500])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [5]:
# prompt: get a list of all unique characters used in text

chars = sorted(list(set(text)))
vocab_size = len(chars)
print('all the unique characters:', ''.join(chars))
print('vocabulary size:', vocab_size)


all the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocabulary size: 65


In [6]:
# prompt: create a mapping from the unique characters found the above cell to integers

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
# prompt: encode all of the text and save it as a tensor

import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:500]) # the first 500 characters will be converted to integers

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
# prompt: split the data into a train and test split, the first 90% should be train and the rest is test.

# Train and test splits
n = int(0.9*len(data)) # first 90% will be train, rest test
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
torch.manual_seed(21123)

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [12]:
batch_size = 4
block_size = 8
TRAIN = 'train'

def get_batch(split):
  data = train_data if split == TRAIN else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i : i+block_size] for i in ix])
  y = torch.stack([data[i+1 : i+block_size+1] for i in ix])

  return x, y


In [13]:
xb, yb = get_batch(TRAIN)

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

when input is [50] the target: 1
when input is [50, 1] the target: 45
when input is [50, 1, 45] the target: 53
when input is [50, 1, 45, 53] the target: 8
when input is [50, 1, 45, 53, 8] the target: 0
when input is [50, 1, 45, 53, 8, 0] the target: 0
when input is [50, 1, 45, 53, 8, 0, 0] the target: 24
when input is [50, 1, 45, 53, 8, 0, 0, 24] the target: 27
when input is [57] the target: 58
when input is [57, 58] the target: 59
when input is [57, 58, 59] the target: 40
when input is [57, 58, 59, 40] the target: 40
when input is [57, 58, 59, 40, 40] the target: 53
when input is [57, 58, 59, 40, 40, 53] the target: 56
when input is [57, 58, 59, 40, 40, 53, 56] the target: 52
when input is [57, 58, 59, 40, 40, 53, 56, 52] the target: 1
when input is [58] the target: 1
when input is [58, 1] the target: 57
when input is [58, 1, 57] the target: 43
when input is [58, 1, 57, 43] the target: 43
when input is [58, 1, 57, 43, 43] the target: 47
when input is [58, 1, 57, 43, 43, 47] the target

In [14]:
print(xb)
print(yb)

tensor([[50,  1, 45, 53,  8,  0,  0, 24],
        [57, 58, 59, 40, 40, 53, 56, 52],
        [58,  1, 57, 43, 43, 47, 52, 45],
        [ 0, 21,  5, 50, 50,  1, 47, 52]])
tensor([[ 1, 45, 53,  8,  0,  0, 24, 27],
        [58, 59, 40, 40, 53, 56, 52,  1],
        [ 1, 57, 43, 43, 47, 52, 45,  6],
        [21,  5, 50, 50,  1, 47, 52,  1]])


In [15]:
import torch.nn as nn
from torch.nn import functional as F

# Building a NN
# Logits = output of a NN before the activation function is applied.
class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    # each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):


    logits = self.token_embedding_table(idx) # (B, T, C)

    # Reshaping to match cross entrophy function requirements.
    # Found here: https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html

    # B = Batch = batch_size = 4
    # T = Time = block_size = 8
    # C = Channel = vocab_size = 65 (Total unique characters in the dataset)
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss


  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      # get the predictions, self(idx) calls the forward function, targets are optional.
      logits, loss = self(idx)
      # focus only on the last time step (as the last value is the prediction)
      logits = logits[:, -1, :] # becomes (B, C)
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      # torch.multinomial helps you randomly pick items based on their chances of being picked.
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index to the running sequence to make it B, T+1
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

    return idx




m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
idx = torch.zeros((1, 1),  dtype=torch.long)


print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.5296, grad_fn=<NllLossBackward0>)

PgDjwM $TbDIheuhkSFAQUInQufgWOWef;xInx;atKvFhyYrACeR!AWOn:QZ. KBV:U?
pW!VuAENOWtqhRrJZEsPk.xaQKD!A
Q


In [16]:
 # create a Pytorch optimizer
 # lr = learning rate = The learning rate is a scalar value (typically between 0 and 1) that dictates how much the model's internal parameters (weights) are adjusted during each training step.
 # it decides how fast the training converges.
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)


In [18]:
batch_size = 32
for steps in range(10000):

  xb, yb = get_batch(TRAIN)

  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.408472776412964


In [19]:
# A huge improvement after training in the above step
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


Y ly we glispudllourt?
Whe
ANo me handu, ithinomatoor'then inHNGERLOTheis t s? l ringot sh y thinove
